# Lab 5: Bayesian Optimization and Surrogate Modeling via Gaussian Processes

### Lab Date: Wednesday, February 26

### Lab Due: Wednesday, March 5th

## Instructions

Work with your lab group to complete the following notebook. Your work will be reviewed by your peers in lab next week (Wednesday, March 5th). 

In this lab, you will:

1. Practice extending the normal, normal model to infinite dimensional regression problems
1. Learn about Gaussian Process (GP) regression and surrogate modeling
1. Learn how to use GP regression to perform Bayesian optimization

The first learning goal illustrates the power of scaleable conjugate models. The second and third demonstrate modern applications enabled by high efficiency Bayesian inference. Together, these act as a capstone demonstration of many of the ideas we've seen so far in the class and many of the advantages of a Bayesian approach: 
- regularization through prior assumptions, 
- probabilistic uncertainty quantification on the unknown, 
- consistent updating in sequential inference, 
- sensible decision theory, and 
- scaleable computation via conjugate modeling!

If you are new to working in python, or in a Jupyter notebook, please ask your lab members for help. If you notice a lab member struggling, and have experience, please offer your help.

Please see this [Ed post](https://edstem.org/us/courses/74615/discussion/6257662) for corrections, questions, and discussion. If you would rather work with your own copy of the files, I have uploaded a zip folder there with the lab materials. 

Corrections to the lab will be pushed directly to this notebook. We will only push corrections to the text, which is set to read only to prevent merge conflicts. In the event of a merge conflict, save your notebook under a different name, and click the link that launches the lab from the schedule on the [stat238 homepage](https://stat238.berkeley.edu/spring-2025/) again. Then, check for discrepancies. If you can't find them, or resolve the conflict, contact us.

In [1]:
# load whatever packages you prefer here. We've added a reference list here
import numpy as np
import scipy as scipy
from scipy import linalg
import matplotlib.pyplot as plt

### Part I: Gaussian Process Regression

#### Background:

In your last Lab, you saw that the normal, normal model extends easily to high-dimensional problems. In this lab we will study the limiting generalization of the normal, normal model... *infinite dimensional* unknowns. In particular, we will focus on the problem of recovering an unknown function $f: \mathcal{X} \rightarrow \mathbb{R}$ given a finite set of sample nodes $\vec{x} = \{x_j\}_{j=1}^n$ and noisy samples at the nodes, $\vec{y} = [f(x_1),f(x_2),f(x_3),...,f(x_n)] + \zeta$ where $\zeta \sim \mathcal{N}(0,C_{y})$. 

You may recognize this as an interpolation or fitting problem. In the standard narrative, we restrict $f$ to a finite-dimensional set of functions with $m < n$ degrees of freedom, often by expressing $f$ in a parametric form (say, a linear combination of feature/basis functions), then by solving for the MLE parameters. 

Stochastic processes offer a non-parametric alternative. Instead of restricting $f$ to a low-dimensional function space, we can pose a prior distribution over the set of all possible functions $f$ by assuming that $f$ is sampled from a stochastic process. Given a prior, we can condition on the noisy samples $\vec{y}$ to produce a posterior distribution over the possible functions $f$. This posterior can be used for Bayesian inference. For example identify the posterior distribution over the value of $f$ at a finite set of query points, interval estimates for the values of $f$ at a finite set of query points, or specify a posterior point estimator for $f$ (either the posterior mode or mean). Importantly, by varying the stochastic process, we can vary what characteristics of $f$ we promote in our inferences, and which we discourage. 

Gaussian processes (GP's) are a popular, expressive, class of stochastic processes. A GP is a stochastic process with the property that *any set of finitely many samples of the process are multivariate normally distributed.* White noise, colored noise, auto-regressive processes with Gaussian updates of any order, Brownian motion, integrated Brownian motion, draws from stochastic differential equations, and all linear combinations of a fixed set of basis functions with Gaussian coefficients are GPs. The following summary is based on Rasmussen and Williams, *Gaussian Processes for Machine Learning*, Chapter 2. You can access select chapters through the course references listed on [Ed](https://edstem.org/us/courses/74615/discussion/6002002). 

A GP is parameterized by a mean function $\mu:\mathcal{X} \rightarrow \mathbb{R}$ and a covariance function $k:\mathcal{X} \times \mathcal{X} \rightarrow \mathbb{R}$ where, given:

$$
f \sim \text{GP}(\mu,k)
$$

the mean and covariance functions satisfy:

$$
\mathbb{E}[f(x)] = \mu(x) \text{ and } \text{Cov}[f(x),f(x')] = k(x,x').
$$

Note, the covariance function must be chosen so that it produces symmetric positive semi-definite matrices when evaluated at any finite set of sample nodes. Often, we assume that the process is stationary (its distribution is translationally symmetric). Then, the covariance function may be expressed with a kernel, $\kappa$, where $k(x,x') = \kappa(x - x')$. If, in addition, we assume that the process is isotropic, then its distribution is symmetric under all unitary coordinate transformations, so $K(x,x') = \kappa(\|x- x'\|)$. 

Popular choices for $\kappa$ include the squared exponential kernel, $\kappa(r) = \sigma_f \exp(-\frac{1}{2}\frac{r}{l}^2)$, the exponential kernel, $\kappa(r) = \sigma_f \exp(-\frac{|r|}{l})$, or the Matern kernel, which interpolates between the exponential and squared exponential kernels. The parameter $\sigma_f$ sets the variance in $f(x)$ at each input $x$. The parameter $l$ is interpreted as a smoothness length scale. The larger $l$, the slower the kernels decay, so the stronger the correlation between the values of $f$ at $x$ and $x'$. 

The choice of kernel controls the behavior of the functions sampled from the posterior, and the posterior estimators. In general, the squared exponential kernel leads to highly regular functions (samples are almost surely continuously differentiable to all orders, analytic, and posterior estimators are drawn from the reproducing kernel Hilbert space built from linear combinations of the kernel function) while the exponential kernel leads to highly rough functions (samples are Markovian, posterior trajectories are nowhere differentiable, and posterior estimators are piecewise linear). By adopting a Matern kernel, it is possible to vary the properties of the posterior ensemble between these two extremes (e.g. to generate posteriors whose samples are almost surely differentiable up to a finite degree). For more details on kernel modeling, see [Rasmussen and Williams Chapter 4](https://edstem.org/us/courses/74615/discussion/6002002).

Now, consider the joint model:

$$
\begin{aligned}
& \textbf{Prior: } f \sim \text{GP}(\mu,k) \\
& \textbf{Measurement: } \vec{y} = f(\vec{x}) + \zeta, \text{ where } f(\vec{x}) = [f(x_1),f(x_2),...,f(x_n)] \\
& \textbf{Noise: } \zeta  \sim \mathcal{N}(0,\sigma_y^2 I_{n \times n})
\end{aligned}
$$

for any finite set of sample nodes $\vec{x}$.

Since $f$ is drawn from a GP, $f(\vec{x})$ is multivariate normally distributed with mean $\mu(\vec{x})$ and covariance $K_{\vec{x},\vec{x}}$, the matrix with $i,j$ entries $K(x_i,x_j)$. Then, since the noise is Gaussian with known covariance, this model is, for all finite sets of sample nodes, a normal, normal model!

This model can be easily extended to include a set of query nodes where we would like to infer the value of $f$. Let $\vec{q} = [q_1,q_2,...,q_m]$ denote a collection of $m$ query nodes, $q_j \in \mathcal{X}$. Let $\vec{c} = [\vec{x},\vec{q}]$ denote the vector formed by composing the sample nodes and query nodes. Then, since $f$ is drawn from a GP, the vector $f(\vec{c})$ is also mutlivariate normally distributed with:

$$
f(\vec{c}) \sim \mathcal{N}(\mu(\vec{c}),K_{\vec{c},\vec{c}}) \text{ where } \mu(\vec{c}) = [\mu(\vec{x}),\mu(\vec{q})] \text{ and } K_{\vec{c},\vec{c}} = \left[\begin{array}{cc} K_{\vec{x},\vec{x}} & K_{\vec{x},\vec{q}} \\ K_{\vec{q},\vec{x}} & K_{\vec{q},\vec{q}} \end{array}\right].
$$

The covariance matrix for the composite vector can be computed by evaluating its blocks:

$$
[K_{\vec{x},\vec{x}}]_{i,j} = k(x_i,x_j), \quad [K_{\vec{x},\vec{q}}]_{i,j} = k(x_i,q_j), \quad [K_{\vec{q},\vec{q}}]_{i,j} = k(q_i,q_j)
$$

Then, we can represent our measurement $\vec{y} = M f(\vec{c})$ where $M$ is a block matrix of the form $M = \left[\begin{array}{cc} I_{n \times n} & 0  \end{array}\right]$

Now, we have translated our original problem into the standard normal, normal form. Estimating $f$ at the query points $\vec{q}$ is the same as performing posterior inference on the unknown quantities $f(\vec{q})$ given:

$$
\begin{aligned}
& \textbf{Prior: } f(\vec{c}) \sim \mathcal{N}(\mu(\vec{c}),K_{\vec{c},\vec{c}}) \\
& \textbf{Measurement: } \vec{y} = M f(\vec{c}) + \zeta \\
& \textbf{Noise: } \zeta \sim \mathcal{N}(0,\sigma_y^2 I_{n \times n}). 
\end{aligned}
$$

Then, $f(\vec{c}), \vec{y}$ are jointly normally distributed, so the posterior $f(\vec{c})|\vec{y}$ is also normally distributed, with mean and covariance determined by the usual update formulas we derived for the normal, normal model.

Simplifying yields the posterior:

$$
f(\vec{q})|\vec{y} \sim \mathcal{N}(\hat{f}_{|\vec{y}}(\vec{q}),C_{f(\vec{q})|\vec{y}})
$$

where:

$$
\begin{aligned}
& \hat{f}_{|\vec{y}}(\vec{q}) = \mu(\vec{q}) + K_{\vec{q},\vec{x}} (K_{\vec{x},\vec{x}} + \sigma_y^2 I_{n \times n})^{-1} (\vec{y} - \mu(\vec{x})) \\
& C_{f(\vec{q})|\vec{y}} = K_{\vec{q},\vec{q}} - K_{\vec{q},\vec{x}} (K_{\vec{x},\vec{x}} + \sigma_y^2 I_{n \times n})^{-1} K_{\vec{x},\vec{q}} \\
\end{aligned}
$$

Notice that we now have a machine that can return a posterior estimate for the unknown function at any finite set of sample points, and, the posterior distribution at that set of samples if multivariate normal. It follows that, given a GP prior, and linear measurements with Gaussian noise, the posterior process is *also a GP!* In particular, the posterior ensemble corresponds to the GP:

$$
f_{|\vec{y}} \sim \text{GP}(\mu_{|\vec{y}},k_{|\vec{y}})
$$

with mean and covariance functions:

$$
\begin{aligned}
& \mu_{|y}(x) = \mu(x) + k(x,\vec{x})(K_{\vec{x},\vec{x}}  + \sigma^2 I_{n \times n})^{-1} (\vec{y} - \mu(\vec{x})) \\
& k_{|y}(x,x') = k(x,x') - k(x,\vec{x}) (K_{\vec{x},\vec{x}} + \sigma^2 I_{n \times n})^{-1} k(\vec{x},x').
\end{aligned}
$$

Here we see a conjugate model over an infinite dimensional function space. Given a GP prior, and linear measurements with Gaussian noise, the posterior is also a GP. 

The figure below ([source](https://en.wikipedia.org/wiki/Gaussian_process#/media/File:Gaussian_Process_Regression.png)) illustrates a 1-dimensional example. The left most panel illustrates sample functions drawn from the prior. The middle panel shows samples drawn from the posterior given a set of (noise-less) observations. Notice that there are many different possible functions that could pass through the observations. The rightmost panel summarizes the posterior distribution by plotting the expected value of the unknown function at each possible query location (solid blue line), and by drawing a shaded region representing a credible interval at each possible query location (grey shaded region). The dashed blue line in the left most panel illustrates a sample from the posterior. Notice that the sample is a reasonable interpolant.

![GP regression example (Wikipedia)](Gaussian_Process_Regression_Wikipedia_Demo_Image.png "GP Regression")

Usually, the posterior is summarized by:
1. Returning the posterior mean estimator $\hat{f}_{|\hat{y}}(x) = \mathbb{E}[f(x)|\vec{y}] = \mu_{|\vec{y}}(x)$. Notice that, since the posterior distribution is Gaussian at each possible sample point, the posterior mean estimator is also the MAP estimator.
1. Returning a credible interval for the value of the unknown function at each query point, $I_{f|\vec{y}}(x) = \mu_{|\vec{y}}(x) \pm s \sqrt{k_{|\vec{y}}(x,x)}$ where the number of standard deviations, $s$, is adopted so that the credible interval covers a chosen fraction of the sample function draws. These are related by the standard percentiles of the Gaussian distribution. Usually, $s = 2$ or 3 to achieve 95\% or 99.7\% coverage.

#### Implementation:

**Q 1.1:** In the space below write a code that can compute the posterior mean and covariance functions for any set of query points given $\mathcal{X} = \mathbb{R}$, an arbitrary prior mean function, and a stationary covariance with an arbitrary kernel $k(x,x') = \kappa(|x - x'|)$.

In [ ]:
# Insert your code here

**Q 1.2:** In the space below, write a pair of functions, one that implements the squared exponential kernel, and one that implements the exponential kernel.

In [ ]:
# Insert your code here

**Q 1.3:** Write a code that, given the mean and covariance function for a GP can:
1. Plot the posterior mean estimator at a set of query points
1. Create a shaded region showing the credible interval with 95\% coverage at a set of query points
1. Plot the 99\% lower and upper credible bounds on the unknown function value at a set of query points

In [ ]:
# Insert your code here

**Q 1.4:** Write a code that can generate samples from a GP with a known mean and covariance function at an input set of sample points, and can plot the associated sample trajectories. 

In [ ]:
# Insert your code here

**Q 1.5:** Suppose that $\vec{x} = [-5, 0, 1, 3, 10]$ and $\vec{y} = [1, 10, 9, 11, 6]$. Generate plots showing the posterior mean estimator, 95\% credible interval, 99\% upper and lower credible bounds, and 3 sample functions for 1,000 equally spaced query points between -8 and 12. For example visualizations see Figure 2.2 in [Rasmussen and William Chapter 2](https://edstem.org/us/courses/74615/discussion/6002002).

Start with $\mu = 0$, $\sigma_f = 15$, $l = 1$, and $\sigma_y = 0.2$. Compare the results using a squared exponential and exponential kernel by creating two side-by side plots illustrating the difference in the posterior distribution over possible $f$. Comment on the differences you observe.

In [ ]:
# Insert your code and discussion here

**Q 1.6:** Recreate the plots you generated above, but try setting:
1. $\sigma_y = 0$ and $\sigma_y = 8$.
1. $l = 0.25$ and $l = 5$.

Comment on how the posterior over $f$ changes as the likelihood changes (varying $\sigma_y$), and as the prior changes (varying the smoothness length scale $l$). Try to justify any phenomena you observe.

In [ ]:
# Insert your code and discussion here

### Part II: Surrogate Modeling and Bayesian Optimization

#### Background: 

Gaussian processes are particularly popular for *surrogate modeling*.

A surrogate model is a statistical model introduced to roughly approximate a more complicated underlying model, often, an unknown objective function. Typically the surrogate model should be fast to evaluate. 

Here's a common setting:

You are asked to optimize an unknown function $f$ over some space of possible inputs $\mathcal{X}$. You have access to an experimental procedure that returns noisy estimates of $f$ at any sample input $x$, however, *each measurement is very expensive*. 

In simulated situations, $f$ can be evaluated by running a complex simulation, but the simulation is very slow, and if stochastic, returns noisy outcomes. Examples include complicated climate models, engineering models that depend on complex PDE solvers (e.g. tokamak design, aero or hydrodynamic simulation), biophysical models that involve extensive Monte Carlo simulation involving many interacting particles (e.g. protein folding simulations), physical models that involve complicated geometries and particle cascades (e.g. particle accelerator design), or physical chemistry models for materials design (e.g. designing materials for carbon capture). In a physical setting, $f$ may represent the idealized outcome of an experiment that may take weeks to run, and will cost real money to implement. 

In all of these cases, it would be useful to have a fast, non-parameteric statistical model that can stand in as a surrogate for running real experiments, that can easily adapt to account for new measurements, and that can be used to guide new experiments.

Gaussian processes check all these boxes. They are fast to evaluate and update using the conjugate formalism developed above, nonparameteric, and, in many cases, sufficiently expressive. Their specification is also reasonably intuitive, so allows interpretable modeling. This is particularly important in the surrogate setting where we only aim for a weakly informative prior. Typically the surrogate is chosen so that sample draws from the surrogate model have the same regularity properties as the unknown objective (i.e. are continuous and sufficiently differentiable). This usually fixes the choice of kernel. They are also typically chosen so that, in absence of information, they will systematically under-estimate (for maximization) or over-estimate (for minimization) the maxima or minima of the objective. The standard $\sigma_f$ is usually chosen to mildly over-estimate the range of possible objective function values, while the length scale $l$ is usually chosen based on a rough guess at the roughness of the function (e.g. some guess at a Lipschitz constant/bound on its derivatives).

Bayesian optimization aims to select informative measurements by leveraging a surrogate model. The surrogate model is updated after each new measurement, and is used to propose new measurements. In general, we assume that the measurement procedure is vastly more expensive than any computational procedure applied to the surrogate model. So, you may assume that the cost of the method is proportional to the number of true function evaluations it demands (sample efficiency). 

These procedures are designed to balance an exploration-exploitation trade-off. We want to propose new measurements that simultaneously provide as much information as possible about the optimal input, while validating the quality of proposed inputs by showing that $f(x)$ is large at any $x$ proposed as a candidate optimizer. 

Two standard procedures are outlined below:

1. **Thompson Sampling:** This approach adopts a greedy randomized heuristic; *"Select the input that optimizes a draw from the posterior."* After each observation, sample a trajectory from the GP posterior, then select the input point that maximizes the sampled trajectory. This can be accomplished by fixing a set of query points, and evaluating the posterior sample at those query points, or by optimizing the sampled trajectory with a standard optimization routine. 
   
1. **Upper-Credible Bound (UCB):** This approach adopts a greedy optimist heuristic; *"Select the input $x$ for which $f(x)$ could plausibly be the largest."* UCB is the reference/standard method for Bayesian optimization.

To implement UCB, define the upper confidence bound function $b(x,p)$ where: 

$$b(x,p) = \inf\{b | \text{Pr}(f(x) < b) \geq p \}$$ 

for some success probability $p$. Notice that, if the surrogate model uses a GP prior, and assumes Gaussian measurement noise, then $b(x,p) = \mu_{|\vec{y}}(x) + s(p) \sqrt{k_{|\vec{y}}(x,x)}$ where $s(p)$ is the $p$ percentile of the standard normal distribution. The first term rewards inputs $x$ which are expected to return large $f$. The second rewards $x$ where $f$ is poorly resolved. 

To select an input, search for the input $x_*(\vec{y},p) = \text{argmax}_{x \in \mathcal{X}}(b(x,p))$. This optimization problem is ugly since $b(x,p)$ is usually a poorly behaved function of $x$. However, evaluating the surrogate model is cheap, so evaluating $b$ is cheap. Usually, we adopt $p > 0.5$ and let $p$ increase towards 1 with each subsequent observation. This ensures that, as the number of measurements increase, we demand increasing certainty in our upper confidence bounds.


**Q 2.1:** With your group, propose a loss function on the inferred function value and the true function value, whose posterior risk is minimized by setting the estimator equal to $b(x,p)$ (see HW 2). Explain why this loss function is sensible for an iterative optimization procedure that selects the next sample point by optimizing the estimated function value. Think carefully about what happens if you underestimate the true function value vs overestimate the function value.

*Insert your answer here.*

#### Demonstration:

Bayesian optimization is particularly useful for highly multimodal objective functions in low-dimensional, or constrained $\mathcal{X}$, since it is based on global optimization of the surrogate. When $\mathcal{X}$ is low-dimensional, it is possible to exhaustively optimize the surrogate, and thus, to suggest sequences of inputs that converge to the global minimizer of the true objective $f$ without getting stuck in local minima. In this part, you'll test this idea.

To illustrate the efficacy (or inefficacy) of Bayesian optimization using GP surrogates, consider the following three [standard test functions](https://www.sfu.ca/~ssurjano/optimization.html). In all cases, we seek to find the global minimizer of these functions.

1. The [1D Levy function](https://www.sfu.ca/~ssurjano/levy.html): $\mathcal{X} = [-10,10]$ where: $$f(x) = \sin^2 (\pi w) + (w - 1)^2 (1 + 10 \sin^2(\pi w + 1) + (w - 1)^2 (1 + \sin^2(2 \pi w)$$ for $w = 0.75 + 0.25 x$. The Levy function has a minimum of 0 at $x_* = 1$.
1. The [2D Ackley function](https://www.sfu.ca/~ssurjano/ackley.html): $\mathcal{X} = [-32.768,32.768] \times [-32.768,32.768]$ where: $$f(x) = 20 \exp\left(-0.2 \sqrt{\frac{1}{2} \sum_{j=1}^d x_j^2} \right) - \exp\left(\frac{1}{2} \sum_{j=1}^2 \cos(2 \pi x_j) \right) + 20 + e$$ whose global minimum is 0 at $x_* = [0,0]$.
1. The [2D Michalwicz function](https://www.sfu.ca/~ssurjano/michal.html): $\mathcal{X} =[0,\pi] \times [0,\pi]$ where: $$f(x) = - \sum_{j=1}^2 \sin(x_j) \sin\left(\frac{j}{\pi} x_j^2 \right)^{10}$$ whose global minimum is -1.8 at $x_* = [2.20,1.57]$.

**Q 2.2:** In the space below implement each of the three test functions. *Note: to complete the lab, you will only need the Levy function. The other pair of test functions are optional, to be used to complete a mini project.*

In [ ]:
# Insert code here

**Q 2.3:** Based on your experiments in part 1, and inspection of the test functions (follow the links above) propose a GP surrogate. Choose between an exponential or square exponential kernel, suggest a choice for the length scale $l$, and standard deviation $\kappa(0) = \sigma_f$. 

Then, suggest an alternate surrogate choice that you expect to perform poorly. Think about the regularizing effect of the prior model when selecting your surrogates, and how the biases incurred by the prior interact with the optimization routine.

*Propose your surrogates here. Explain why you think the first will perform well, and why the second will perform poorly.*

**Q 2.4:** In the space below prepare two separate codes, one that implements UCB, and one that implements Thompson Sampling for a true objective $f$, domain $\mathcal{X}$, noise level $\sigma_y^2$, kernel function $\kappa$, and smoothness length scale $l$. 
- You may assume that $\mu = 20$ for the Levy and Ackley functions, and $\mu = 0$ for the Michalwicz function. These are intentionally pessimistic choices that must be disproved by near-to-optimal samples.
- You may assume that $\mathcal{X}$ is an interval in one-dimension, or a rectangular region in two-dimensions
- To optimize over query points simply evaluate the acquisition function (either the sampled surrogate function, or the credible bound) on a finely spaced grid. If multiple sample points return the same acquisition value, pick between them uniformly at random. More sophisticated methods use an adaptive grid that adapt to a partitioning of $\mathcal{X}$ induced by the current set of sampled nodes. 
- To initialize, select a sample point uniformly at random.

In [ ]:
# Insert code here

**Q 2.5:** Apply UCB (*Lower*-Credible Bound for minimization), and Thompson sampling to minimize the Levy function using your chosen kernel and best guess at the smoothness parameter $l$. Terminate after 160 iterations. Use your code from Part 1 to display the posterior distribution used for selecting the next sample node after 10, 20, 40 and 160 iterations. Set $\sigma_y = 0$.

In [ ]:
# Insert code here

**Q 2.6:** Repeat 2.5 but try increasing $\sigma_y$ to 0.1, then to 1. Only produce plots for iterations 10, 40, and 160. Comment on how the behavior of the optimizer changes.

In [ ]:
# Insert code here

*Completing the subsequent parts for all three test functions is optional, and will contribute to the completion of a project mini. For the lab, you need only complete the subsequent tasks for the Levy test function.*

**Q 2.7:** Apply UCB (LCB for minimization), and Thompson sampling to minimize each of the three test functions using your preferred surrogate model. Return two sets of three (*if you run all three tests*) plots: 
- Make three adjacent plots, one per objective, showing the locations of the sample nodes selected for evaluation. Use a scatter plot, and color the nodes according to their iteration number (e.g. blue for early iterations, red for late iterations), and change the marker to distinguish UCB from Thompson. Jitter the points slightly so that it is possible to distinguish multiple evaluations of the same sample node. For the Levy function, overlay the scatter cloud on a plot of the true objective. For the 2D test functions, overlay the scatter on a heatmap showing the true objectives. Set the transparency of the heat map to 0.5 or less so that the scatter points are clearly visible.

- Make three comparison plots, one per objective, showing the sequence of sampled objective function values (scatter), and the running minimum of the sampled objective function values (line) under each optimization rule. Your plots should include three panels, one per test function. Overlay the sampled values, and running minimums, for each approach. Add a legend to distinguish Thompson sampling from UCB. Set the minimum value of the vertical axis to the true minimal value of each test function. For this experiment you may use $\sigma_y = 0$.

- Stop the optimization process after, at most, 160 iterations. You may terminate earlier if the value of the sampled objective functions converge near the true minimum values provided above.

In [ ]:
# Insert code here

**Q 2.8:** Repeat the same process but select means $\mu$ that underestimate the desired minima (say $\mu = -0.2$ for the Levy and Ackley functions, and $\mu = -2$ for the Michalwicz function). How does the behavior of the Bayesian optimizer change? (Hint: consider how it prioritizes exploration vs exploitation when $\mu$ underestimates the minimizer.)

In [ ]:
# Insert code here

**Q 2.9:** Repeat the same process, but using the surrogate choice you expected to perform poorly. Comment on the result. 

In [ ]:
# Insert code here

**Q 2.10:** Comment on the efficacy of your Bayesian optimizers in the test cases proposed here. Focus on sample efficiency (the number of samples of $f$ that are evaluated) to evaluate quality and discuss the sensitivity of the method to the choice of surrogate model. Do you think it would be easy to select a suitable surrogate in practical problems? What principles for prior modeling would guide its selection?

*Insert commentary here.*